## Part III (Static Replication)

Suppose on 30-Aug-2013, we need to evaluate European derivatives expiring on 17-Jan-2015 and paying:

- St^3 * 1e-8 + 0.5 * log(St) + 10

Determine the price of these 2 derivative contracts if we use:

- Black-Scholes model (what sigma should we use?)

- 2 Bachelier model (what sigma should we use?)

In [13]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
from scipy import interpolate
import datetime as dt
from scipy.integrate import quad

In [2]:
#Pay-off function using Black Scholes Model
def BS_Payoff(S, K, r, sigma, T):
    A = 1e-8 * (S * np.exp(T * (r+sigma**2)))**3
    B = 0.5 * (np.log(S) + (r - 0.5*sigma**2)*T)
    return np.exp(-r*T) * (A + B + 10)

#Pay-off function using Bachelier Model (assuming r = 0)
def Bach_Payoff(S, K, sigma, T):
    
    # Calculate Expectation by integration log(1 + σT^0.5 X), where  X ~ N(0,1)
    def f(x):
        return (1/np.sqrt(2*np.pi))*np.exp(-0.5*x**2)*np.log(1+sigma*np.sqrt(T)*x)
    lower_bound = -1/(sigma*np.sqrt(T))
    exp_log = quad(f,lower_bound, float('inf'))[0]
    
    A = 1e-8 * (S**3 * (1+3*sigma**2*T))
    B = 0.5 * (np.log(S)+exp_log)
    return A + B + 10

In [3]:
# Pay-off function using Bachelier Model using Taylor explansion
# def Bach_Payoff(S, K, sigma, T):
#     A = 1e-8 * (S**3 * (1+3*sigma**2*T))
#     B = 0.5 * (np.log(S)-(1/2)*sigma**2*T) #Taylor Expansion to the second order
#     return A + B + 10

In [4]:
#Black-Scholes Model

def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def BlackScholesPut(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

#Implied Call & Put Volatility
def impliedCallVolatility(S, K, r, price, T):
    impliedVol = brentq(lambda x: price -
                        BlackScholesCall(S, K, r, x, T),
                        1e-6, 1)
    return impliedVol

def impliedPutVolatility(S, K, r, price, T):
    impliedVol = brentq(lambda x: price -
                        BlackScholesPut(S, K, r, x, T),
                        1e-6, 1)
    return impliedVol

In [5]:
#Read Data
    
discount_df = pd.read_csv('discount.csv')
call_df = pd.read_csv("goog_call.csv")
call_df['mid_price'] = (call_df['best_bid'] + call_df['best_offer'])/2
put_df = pd.read_csv("goog_put.csv")
put_df['mid_price'] = (put_df['best_bid'] + put_df['best_offer'])/2

In [6]:
discount_df.head()

,Day,Rate (%)
0,7,0.14981
1,14,0.17250
2,19,0.17595
3,47,0.23118
4,82,0.25150


In [7]:
call_df.head()

,date,expiry,strike,best_bid,best_offer,mid_price
0,20130830,20150117,320,525.3,528.8,527.05
1,20130830,20150117,340,505.8,509.2,507.50
2,20130830,20150117,350,496.0,499.2,497.60
3,20130830,20150117,360,486.3,489.4,487.85
4,20130830,20150117,370,476.5,479.7,478.10


In [8]:
put_df.head()

,date,expiry,strike,best_bid,best_offer,mid_price
0,20130830,20150117,320,0.90,2.00,1.450
1,20130830,20150117,340,0.15,2.30,1.225
2,20130830,20150117,350,0.80,2.45,1.625
3,20130830,20150117,360,0.60,2.65,1.625
4,20130830,20150117,370,0.65,2.85,1.750


In [9]:
#Interpolation interest rate
x = discount_df['Day']
y = discount_df['Rate (%)']
f = interpolate.interp1d(x,y)

In [10]:
#Parameters

n = len(call_df.index)
strike = call_df['strike'].values

begin = dt.date(2013, 8, 30)
expiry = dt.date(2015, 1, 17)

T = (expiry-begin).days/365
S = 846.9
r = f(T*365)/100
K = 850

#Calculate at the money sigma
atm_call = (100+102.8)/2
atm_put = (101.8+104)/2
sigma_call = impliedCallVolatility(S, K, r, atm_call, T)
sigma_put = impliedPutVolatility(S, K, r, atm_put, T)
sigma = (sigma_call + sigma_put)/2
print(sigma)

0.25827414878228977


In [11]:
print("T = ",T," S = ", S," r = ",r," Sigma = ",sigma)

T =  1.3835616438356164  S =  846.9  r =  0.004053595604395604  Sigma =  0.25827414878228977


In [14]:
#Calculate Pricing
BS_Price = BS_Payoff(S, K, r, sigma, T)
print('Black-Scholes Model Price:', BS_Price)
print()
print()
Bach_Price = Bach_Payoff(S, K, sigma, T)
print('Bachelier Model Price:', Bach_Price)

Black-Scholes Model Price: 21.37823513954841


Bachelier Model Price: 21.09949114999562
